# Projet de Simulation et Monte Carlo  
  
  Ce projet se base sur l'arcticle de Yuyang Shi et Rob Cornish : On Multilevel Monte Carlo Unbiased Gradient Estimation For Deep Latent Variable Models (2021) http://proceedings.mlr.press/v130/shi21d.html

In [2]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import random

## Partie 1 : Génération des données 
Comme dit dans l'article, nous étudions les modèles à variables latentes. pour pouvoir générer les données, nous considérerons une expérience gaussienne linéaire de la même manière que Rainforth et al. (2018) et Tucker et al. (2019)  

   On considère le modèle génératif suivant : $p_{\theta}(x,z)= \mathcal{N}(z|\theta,I)\mathcal{N}(x|z,I)$ où $x,z \in \mathbb{R^{20}}$ tel que $p_{\theta}(x)= \mathcal{N}(x|\theta,2I)$ et $p_{\theta}(z|x)= \mathcal{N}(\frac{\theta+x}{2},\frac{1}{2}I)$. La distribution de l'encodeur est $q_{\phi}(z|x)=\mathcal{N}(z|Ax+b,\frac{2}{3}I)$, où $\phi = (A,b)$. On considère une perturbation aléatoire des paramètres autour de la valeur optimale par une gaussienne centré de variance 0.01.
    

In [100]:
theta = np.random.normal(0,1)
print('theta = ' + str(theta))
r = 0.6
K = np.random.geometric(0.6)
print('K = ' + str(K))

theta = -0.40801985026066095
K = 1


In [103]:
A_true = 1/2*np.eye(20)
b_true = np.zeros(20)+theta/2
print(b_true)

[-0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993
 -0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993
 -0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993 -0.20400993
 -0.20400993 -0.20400993]


In [104]:
A = A_true + np.eye(20)*np.random.normal(0,0.01)
b = b_true + np.random.normal(0,0.01)
print(b)

[-0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956
 -0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956
 -0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956 -0.19397956
 -0.19397956 -0.19397956]


In [113]:
def q_phi(x):
    return np.random.normal(A*x+b,2/3*np.eye(20))

def p_joint(z,mean):
    return np.random.normal(mean,np.eye(20))*np.random.normal(z,np.eye(20))

In [114]:
X = np.random.normal(np.zeros(20)+theta, 2*np.eye(20))

array([[-3.95188985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985],
       [-0.40801985, -2.13566566, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985],
       [-0.40801985, -0.40801985,  0.22818886, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -0.40801985, -0.40801985],
       [-0.40801985, -0.40801985, -0.40801985,  3.66208728, -0.40801985,
        -0.40801985, -0.40801985, -0.40801985, -

In [110]:
def generate_z(X_data,k):
    return Z
print(generate_z(X,K))

[array([[-1.39544431, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956],
       [-0.19397956, -0.07336672, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956],
       [-0.19397956, -0.19397956,  0.40402442, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, -0.19397956, -0.19397956],
       [-0.19397956, -0.19397956, -0.19397956,  0.00868066, -0.19397956,
        -0.19397956, -0.19397956, -0.19397956, 

## Partie 2 : Estimation de la vraisemblance  
  
  Dans cette partie, nous allons reproduire les estimateurs IWAE, SS, RR et SUMO cités dans l'article

### 1) Estimateur SS

In [ ]:
def SS_estimator()